In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr
from sklearn.manifold import trustworthiness

# Extract the relative abundance data (assuming columns from 10th onward are abundance data)
abundance_data = data.iloc[:, 10:]

# Dimensionality reduction methods
pca = PCA(n_components=2).fit_transform(abundance_data)
tsne = TSNE(n_components=2).fit_transform(abundance_data)
umap = UMAP(n_components=2).fit_transform(abundance_data)
# VAE implementation would go here, assuming it outputs a 2D array `vae`

# Store the results
embeddings = {'PCA': pca, 't-SNE': tsne, 'UMAP': umap}  # Add 'VAE': vae if implemented

# Clustering and evaluation
results = {}
for method, emb in embeddings.items():
    kmeans = KMeans(n_clusters=3).fit(emb)  # Adjust the number of clusters as needed
    labels = kmeans.labels_
    
    silhouette = silhouette_score(emb, labels)
    davies_bouldin = davies_bouldin_score(emb, labels)
    calinski_harabasz = calinski_harabasz_score(emb, labels)
    
    results[method] = {
        'Silhouette Score': silhouette,
        'Davies-Bouldin Index': davies_bouldin,
        'Calinski-Harabasz Index': calinski_harabasz
    }

# Trustworthiness
trustworthiness_scores = {method: trustworthiness(abundance_data, emb, n_neighbors=5) for method, emb in embeddings.items()}

# Mantel test
def mantel_test(dist1, dist2):
    return pearsonr(dist1.ravel(), dist2.ravel())

dist_original = squareform(pdist(abundance_data))
mantel_results = {method: mantel_test(dist_original, squareform(pdist(emb))) for method, emb in embeddings.items()}

# Print results
results_df = pd.DataFrame(results)
print("Clustering Quality Metrics:")
print(results_df)

print("\nTrustworthiness Scores:")
print(trustworthiness_scores)

print("\nMantel Test Results:")
print(mantel_results)
